In [83]:
import sys 
sys.path.append("/home/eddie_akers_anchorlabs_com/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append("/home/eddie_akers_anchorlabs_com/anchorage/source/python/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility
import utils as aql_utils
import json
import pandas as pd
import os


In [124]:
import requests
import pandas as pd

# Define your list of cryptos and suffix
cryptos = ['BTC', 'ETH', 'SOL', 'XRP']  # Example cryptos
suffix = '-USDT-SWAP'
symbols = [crypto + suffix for crypto in cryptos]

def fetch_funding_rates(symbols):
    funding_rates = []
    base_url = 'https://www.okx.com/api/v5/public/funding-rate'

    for symbol in symbols:
        try:
            # Fetch funding rate from OKX public API
            response = requests.get(base_url, params={'instId': symbol})
            response.raise_for_status()  # Check if request was successful

            data = response.json()
            
            # Print data for debugging
            print(f"Data for {symbol}: {data}")

            if 'data' in data and len(data['data']) > 0:
                # Extract relevant information
                funding_rate_info = data['data'][0]  # Assuming the first entry is relevant
                
                # Helper function to safely convert values to float
                def safe_float(value):
                    try:
                        return float(value)
                    except (TypeError, ValueError):
                        return None
                
                # Extracting fields with default values if not present
                fundingRate = safe_float(funding_rate_info.get('fundingRate', 0))
                fundingTime = pd.to_datetime(funding_rate_info.get('fundingTime'), unit='ms')
                nextFundingTime = pd.to_datetime(funding_rate_info.get('nextFundingTime', 0), unit='ms')
                minFundingRate = safe_float(funding_rate_info.get('minFundingRate', 0))
                maxFundingRate = safe_float(funding_rate_info.get('maxFundingRate', 0))
                ts = pd.to_datetime(funding_rate_info.get('ts'), unit='ms')

                # Calculate cadence as the difference between nextFundingTime and fundingTime
                if nextFundingTime and fundingTime:
                    cadence = nextFundingTime - fundingTime
                    # Convert cadence to hours and round to the nearest whole number
                    cadence_hours = round(cadence.total_seconds() / 3600)
                else:
                    cadence_hours = None

                fundingRates = {
                    'coin': symbol.replace(suffix, ''),  # Extract the base cryptocurrency name
                    'symbol': symbol,
                    'exchange': 'OKX',
                    'fundingRate': fundingRate,
                    'fundingTime': fundingTime,
                    'nextFundingTime': nextFundingTime,
                    'minFundingRate': minFundingRate,
                    'maxFundingRate': maxFundingRate,
                    'ts': ts,
                    'cadence': cadence_hours
                }

                funding_rates.append(fundingRates)
            else:
                print(f"No data returned for {symbol}")
                funding_rates.append({
                    'coin': symbol.replace(suffix, ''),  # Extract the base cryptocurrency name
                    'symbol': symbol,
                    'exchange': 'OKX',
                    'fundingRate': None,
                    'fundingTime': None,
                    'nextFundingTime': None,
                    'minFundingRate': None,
                    'maxFundingRate': None,
                    'ts': None,
                    'cadence': None
                })
        except Exception as e:
            print(f"Error fetching funding rate for {symbol}: {e}")
            funding_rates.append({
                'coin': symbol.replace(suffix, ''),  # Extract the base cryptocurrency name
                'symbol': symbol,
                'exchange': 'OKX',
                'fundingRate': None,
                'fundingTime': None,
                'nextFundingTime': None,
                'minFundingRate': None,
                'maxFundingRate': None,
                'ts': None,
                'cadence': None
            })

    return pd.DataFrame(funding_rates)

# Fetch the funding rates
funding_rates_df = fetch_funding_rates(symbols)

# Display the DataFrame
print(funding_rates_df)



Data for BTC-USDT-SWAP: {'code': '0', 'data': [{'fundingRate': '0.0000289843340695', 'fundingTime': '1720483200000', 'instId': 'BTC-USDT-SWAP', 'instType': 'SWAP', 'maxFundingRate': '0.00375', 'method': 'current_period', 'minFundingRate': '-0.00375', 'nextFundingRate': '', 'nextFundingTime': '1720512000000', 'premium': '-0.0000717705892986', 'settFundingRate': '0.0000173742544987', 'settState': 'settled', 'ts': '1720464194036'}], 'msg': ''}


/tmp/ipykernel_15392/1625052038.py:37 FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.

/tmp/ipykernel_15392/1625052038.py:38 FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.

/tmp/ipykernel_15392/1625052038.py:41 FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.

Data for ETH-USDT-SWAP: {'code': '0', 'data': [{'fundingRate': '0.0000422519684178', 'fundingTime': '1720483200000', 'instId': 'ETH-USDT-SWAP', 'instType': 'SWAP', 'maxFundingRate': '0.0075', 'method': 'current_period', 'minFundingRate': '-0.0075', 'nextFundingRate': '', 'nextFundingTime': '1720512000000', 'premium': '0.0000384046379445', 'settFundingRate': '0.0000529455345788', 'settState': 'settled', 'ts': '1720464194059'}], 'msg': ''}
Data for SOL-USDT-SWAP: {'code': '0', 'data': [{'fundingRate': '0.0002002796102355', 'fundingTime': '1720483200000', 'instId': 'SOL-USDT-SWAP', 'instType': 'SWAP', 'maxFundingRate': '0.015', 'method': 'current_period', 'minFundingRate': '-0.015', 'nextFundingRate': '', 'nextFundingTime': '1720512000000', 'premium': '-0.0001089878660176', 'settFundingRate': '0.0001472245024564', 'settState': 'settled', 'ts': '1720464194321'}], 'msg': ''}
Data for XRP-USDT-SWAP: {'code': '0', 'data': [{'fundingRate': '0.0000805881470175', 'fundingTime': '1720483200000', 

In [139]:
from datetime import datetime

def post_to_slack(message, webhook_url):
    payload = {
        "text": message
    }
    response = requests.post(webhook_url, json=payload)
    if response.status_code == 200:
        print("Message posted to Slack successfully.")
    else:
        print(f"Failed to post message to Slack. Status code: {response.status_code}")

def check_and_alert(funding_rates_df, alerts_df, slack_webhook_url):
    messages = []

    # Merge funding rates with alert settings on coin
    merged_df = pd.merge(funding_rates_df, alerts_df, left_on='coin', right_on='coin', how='left')

    # Print merged DataFrame for debugging
    # print("Merged DataFrame:")
    # print(merged_df.head())

    for _, row in merged_df.iterrows():
        if row['AlertOn?'] and row['fundingRate'] is not None:
            try:
                lower_bound = float(row['Lower'])
                upper_bound = float(row['Upper'])
                funding_rate = float(row['fundingRate'])
                cadence = row['cadence']

                if cadence and cadence > 0:
                    # Calculate annualized funding rate using the new formula
                    annualized_funding_rate = funding_rate * (24 / cadence) * 365
                    annualized_funding_rate_percentage = annualized_funding_rate * 100

                    # Calculate time until next funding rate
                    if row['fundingTime'] and row['ts']:
                        next_funding_time = pd.to_datetime(row['fundingTime'])
                        current_time = pd.to_datetime(row['ts'])
                        time_until_next_funding = next_funding_time - current_time

                        # Extract hours and minutes from the timedelta
                        total_seconds = time_until_next_funding.total_seconds()
                        hours = int(total_seconds // 3600)
                        minutes = int((total_seconds % 3600) // 60)
                        
                        time_until_next_funding_str = f"{hours} hours, {minutes} minutes"
                    else:
                        time_until_next_funding_str = "N/A"

                    # Determine if the funding rate is out of bounds
                    if funding_rate < lower_bound:
                        status = 'Less Than'
                        comparison = 'Lower Bound'
                    elif funding_rate > upper_bound:
                        status = 'Greater Than'
                        comparison = 'Upper Bound'
                    else:
                        status = None
                        comparison = None

                    if status:
                        message = (f":rotating_light: Alert for *{row['symbol']}!*\n"
                               f"Current Fund Rate Annualized: *{annualized_funding_rate_percentage:.2f}%*\n"
                               f"Time Until Next Funding Rate: {time_until_next_funding_str}\n"
                               f"cc: <!here>")
                        messages.append(message)
                else:
                    print(f"Invalid cadence value: {cadence}")

            except ValueError:
                print(f"Invalid value encountered in row: {row}")

    # Post all messages to Slack if any alerts are triggered
    if messages:
        for message in messages:
            post_to_slack(message, slack_webhook_url)
    else:
        print("No alerts triggered.")

In [ ]:
os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Cornerstone_GS"
tab_name = "fundingRateAlerts"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

alerts_df = gsu._get_current_sheet_df(tab_name, 0)

cryptos = alerts_df['coin'].tolist()

# Define your list of cryptos and suffix
cryptos = cryptos  # Example cryptos
suffix = '-USDT-SWAP'
symbols = [crypto + suffix for crypto in cryptos]

# Fetch the funding rates
funding_rates_df = fetch_funding_rates(symbols)

# Example alerts_df (make sure it contains the actual alert settings)

# Define your Slack webhook URL (replace with your actual webhook URL)
#slack_webhook_url = ''

#To do - fetch value from secret manager

# Check funding rates and send alerts if needed
check_and_alert(funding_rates_df, alerts_df, slack_webhook_url)

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Cornerstone_GS"
tab_name = "fundingRateData"


gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
gsu.dump_current_sheet(tab_name, funding_rates_df)


/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

Data for BTC-USDT-SWAP: {'code': '0', 'data': [{'fundingRate': '0.0000280813574294', 'fundingTime': '1720483200000', 'instId': 'BTC-USDT-SWAP', 'instType': 'SWAP', 'maxFundingRate': '0.00375', 'method': 'current_period', 'minFundingRate': '-0.00375', 'nextFundingRate': '', 'nextFundingTime': '1720512000000', 'premium': '0.0000577544706403', 'settFundingRate': '0.0000173742544987', 'settState': 'settled', 'ts': '1720465155035'}], 'msg': ''}


/tmp/ipykernel_15392/1625052038.py:37 FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.

/tmp/ipykernel_15392/1625052038.py:38 FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.

/tmp/ipykernel_15392/1625052038.py:41 FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.

Data for ETH-USDT-SWAP: {'code': '0', 'data': [{'fundingRate': '0.0000421348748840', 'fundingTime': '1720483200000', 'instId': 'ETH-USDT-SWAP', 'instType': 'SWAP', 'maxFundingRate': '0.0075', 'method': 'current_period', 'minFundingRate': '-0.0075', 'nextFundingRate': '', 'nextFundingTime': '1720512000000', 'premium': '0.0001722016397609', 'settFundingRate': '0.0000529455345788', 'settState': 'settled', 'ts': '1720465155057'}], 'msg': ''}
Data for SOL-USDT-SWAP: {'code': '0', 'data': [{'fundingRate': '0.0001954045493851', 'fundingTime': '1720483200000', 'instId': 'SOL-USDT-SWAP', 'instType': 'SWAP', 'maxFundingRate': '0.015', 'method': 'current_period', 'minFundingRate': '-0.015', 'nextFundingRate': '', 'nextFundingTime': '1720512000000', 'premium': '0.0000362871035634', 'settFundingRate': '0.0001472245024564', 'settState': 'settled', 'ts': '1720465155307'}], 'msg': ''}
Data for XRP-USDT-SWAP: {'code': '0', 'data': [{'fundingRate': '0.0000738637357989', 'fundingTime': '1720483200000', '

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.